In [85]:
##
import os
import pickle
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable, Dict, List

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
# FinRL
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
# FinRL-Meta
from meta.data_processors.yahoofinance import Yahoofinance

##
import yfinance as yf

In [86]:
def config():
    ##
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    ##
    matplotlib.use("Agg")


config()

In [87]:
##
import pandas as pd
import tqdm
from dotenv import load_dotenv
import financedatabase as fd
import fundamentalanalysis as fa

##
from common.utils import now_time
from configuration.settings import ProjectDir
from rl.data.financial_modeling_prep import download
from rl.data.CompanyFund import CompanyFund
from rl.data.CompanyPrice import CompanyPrice

In [88]:
##
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))

##
_TRAIN_DATA_START = "2000-01-01"
_TRAIN_DATA_END = "2015-12-31"

##
_TEST_DATA_START = "2016-01-01"
_TEST_DATA_END = "2021-12-31"

In [89]:
# #################################################################################

In [90]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
load_dotenv(prj_dir.root.joinpath("env/.env"))
try:
    print(os.getenv("EOD_HISTORICAL_DATA_API"))
    print(os.getenv("NASDAQ_HISTORICAL_DATA_API"))
    print(os.getenv("ALPHA_VANTAGE_API"))
    print(os.getenv("FINANCIAL_MODELING_PREP_API"))
except:
    raise ValueError("ERROR: API keys")

639b4cc5af44e7.93519913
B9p3RKb-K6xWwBRLso7Z
G21Y7NB8U8VJFFIH
6b0006554aa86e3e21493713af2af3c4


In [91]:
with open(prj_dir.dataset.indexes.joinpath("DOW_30_companies_2022-12-17T13-59-23.pkl"), "rb") as f:
    data: Dict[str, CompanyFund] = pickle.load(f)

data.keys()

dict_keys(['DIS', 'DOW', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX'])

In [92]:
# Price States = date,tic,open,high,low,close,volume
# Ratio States = OPM,NPM,ROA,ROE,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,PE,PB,Div_yield

ratios = list(
    [
        "operatingProfitMargin",
        "netProfitMargin",
        "returnOnAssets",
        "returnOnEquity",
        "currentRatio",
        "quickRatio",
        "cashRatio",
        "inventoryTurnover",
        "receivablesTurnover",
        "payablesTurnover",
        "debtRatio",
        "debtEquityRatio",
        "priceEarningsRatio",
        "priceBookValueRatio",
        "dividendYield",
    ]
)

In [93]:
aapl_ratios = data["AAPL"].financial_ratios
wanted_ratios = pd.Series(aapl_ratios.index.isin(ratios))
aapl_wanted_ratios = aapl_ratios[wanted_ratios.values]

In [94]:
# print(aapl_wanted_ratios)
aapl_wanted_ratios

,2022-09,2022-06,2022-03,2021-12,2021-09,2021-06,2021-03,2020-12,2020-09,2020-06,...,1992-03,1991-12,1991-09,1991-06,1991-03,1990-12,1990-09,1990-06,1990-03,1989-12
currentRatio,0.879356,0.864629,0.926844,1.037812,1.074553,1.061891,1.141749,1.163003,1.363604,1.46945,...,2.603645,2.41706,2.352806,2.197933,2.82363,2.37247,2.340117,2.637276,2.931293,2.605018
quickRatio,0.709408,0.696627,0.760039,0.875263,0.90966,0.887243,0.96696,1.02218,1.218195,1.312449,...,1.311136,1.183192,1.241722,1.197389,1.671552,1.485405,1.106719,1.54527,1.684877,1.298869
cashRatio,0.153563,0.211761,0.220363,0.251528,0.278449,0.315998,0.361574,0.271759,0.36071,0.350228,...,0.612999,0.417139,0.496344,0.601259,0.877635,0.689453,0.364849,0.749803,0.834044,0.459841
operatingProfitMargin,0.276152,0.278162,0.308179,0.334729,0.285341,0.296264,0.307008,0.300918,0.228369,0.219335,...,0.122611,0.140986,0.080491,-0.058092,0.116793,0.136318,0.103833,0.133573,0.147972,0.127226
netProfitMargin,0.22986,0.234357,0.257098,0.279398,0.246533,0.267014,0.263775,0.258034,0.195879,0.18854,...,0.07873,0.089123,0.053882,-0.034738,0.082055,0.089824,0.072742,0.087778,0.097905,0.083568
returnOnAssets,0.05874,0.05781,0.071322,0.090847,0.05855,0.065923,0.070086,0.081216,0.039128,0.03546,...,0.034282,0.04432,0.023243,-0.015367,0.038309,0.047667,0.033101,0.04146,0.047484,0.044361
returnOnEquity,0.408924,0.33459,0.371074,0.481427,0.325741,0.33827,0.341583,0.434208,0.193958,0.155682,...,0.064734,0.086332,0.045961,-0.031867,0.072255,0.097405,0.068081,0.079506,0.084983,0.081901
debtRatio,0.856354,0.827221,0.807795,0.811297,0.820257,0.805118,0.79482,0.812955,0.798267,0.772228,...,0.470426,0.486637,0.494304,0.517783,0.469815,0.510626,0.513795,0.478526,0.441258,0.458359
debtEquityRatio,5.961537,4.787754,4.202777,4.299324,4.563512,4.131301,3.873775,4.346309,3.957039,3.39036,...,0.888309,0.947941,0.977472,1.073756,0.886133,1.043428,1.056746,0.91764,0.789735,0.84624
receivablesTurnover,1.479453,1.963898,2.142687,1.899453,1.618452,2.401616,2.711708,1.901041,1.727814,1.860795,...,1.96496,1.913303,1.661155,1.9927,2.020104,1.963093,1.777267,1.934515,2.043412,1.993858


In [95]:
# binary = s.str.contains('div', case=False)
# print(binary)
# aapl_ratios[binary.values]

In [96]:
api_key = os.getenv("FINANCIAL_MODELING_PREP_API")

In [97]:
prices: Dict[str, CompanyPrice] = dict()
pbar = tqdm.tqdm(data.keys())
for symbol in pbar:
    pbar.set_description("Symbol: %s" % symbol)
    stock_data_detailed = fa.stock_data_detailed(symbol, api_key)
    prices[symbol] = CompanyPrice(symbol=symbol, prices=stock_data_detailed)

Symbol: CVX: 100%|██████████| 30/30 [02:02<00:00,  4.09s/it] 


In [99]:
prices["AAPL"].prices

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
2022-12-16,136.685,137.65,133.73,134.51,134.51,159808331.0,159808331.0,-2.175,-1.591,135.29667,"December 16, 22",-0.01591
2022-12-15,141.11,141.8,136.03,136.5,136.5,98827150.0,98827150.0,-4.61,-3.267,138.11,"December 15, 22",-0.03267
2022-12-14,145.35,146.66,141.16,143.21,143.21,82234160.0,82234160.0,-2.14,-1.472,143.67667,"December 14, 22",-0.01472
2022-12-13,149.5,149.97,144.24,145.47,145.47,93831797.0,93831797.0,-4.03,-2.696,146.56,"December 13, 22",-0.02696
2022-12-12,142.7,144.5,141.06,144.49,144.49,70422099.0,70422099.0,1.79,1.254,143.35,"December 12, 22",0.01254
...,...,...,...,...,...,...,...,...,...,...,...,...
1980-12-18,0.118862,0.11942,0.118862,0.118862,0.092492,73449600.0,73449600.0,-0.02637,-22.185,0.11026,"December 18, 80",-0.22185
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089886,86441600.0,86441600.0,-0.02563,-22.188,0.10716,"December 17, 80",-0.22188
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087715,105728000.0,105728000.0,-0.02557,-22.572,0.10457,"December 16, 80",-0.22572
1980-12-15,0.12221,0.12221,0.121652,0.121652,0.094663,175884800.0,175884800.0,-0.02755,-22.543,0.11284,"December 15, 80",-0.22543


In [101]:
data["AAPL"]

Company(symbol='AAPL', balance_sheet=                                                                                   2022-09  \
reportedCurrency                                                                       USD   
cik                                                                             0000320193   
fillingDate                                                                     2022-10-28   
acceptedDate                                                           2022-10-27 18:01:14   
calendarYear                                                                          2022   
period                                                                                  Q4   
cashAndCashEquivalents                                                         23646000000   
shortTermInvestments                                                           24658000000   
cashAndShortTermInvestments                                                    48304000000   
netReceivables         